In [1]:
import os

os.environ["WANDB_DISABLED"] = "true"

In [2]:
from gtts import gTTS
import os
import json
import librosa
import soundfile as sf
import numpy as np

def main():
    print("==== AI-Driven Voiceover for Interactive Storytelling ====")

    # Create output directory if it doesn't exist
    if not os.path.exists("voiceovers"):
        os.makedirs("voiceovers")

    # Story input
    print("\nEnter your story text (type 'END' on a new line when finished):")
    story_lines = []
    while True:
        line = input()
        if line == "END":
            break
        story_lines.append(line)

    story_text = " ".join(story_lines).strip()

    # If no text is entered, use a sample
    if not story_text:
        story_text = "This is a sample story for demonstration purposes."
        print("No text entered, using sample text.")

    # Voice selection
    print("\nSelect voice style:")
    print("1. Calm/Relaxed")
    print("2. Neutral")
    print("3. Energetic")

    style_choice = int(input("Enter choice (1-3): "))

    # Language selection
    print("\nSelect language:")
    print("1. English")
    print("2. Spanish")
    print("3. French")
    print("4. German")

    lang_choice = int(input("Enter language choice (1-5): "))

    # Map language choice to language code
    lang_map = {
        1: 'en',
        2: 'es',
        3: 'fr',
        4: 'de'
    }
    lang_code = lang_map.get(lang_choice, 'en-us')

    # Output filename
    output_filename = input("\nEnter output filename (without extension): ")
    temp_path = f"voiceovers/{output_filename}_temp.mp3"
    final_path = f"voiceovers/{output_filename}.mp3"

    print("\nGenerating voiceover (this may take a moment)...")
    print(f"Processing text: \"{story_text}\"")

    # Generate speech with gTTS
    try:
        tts = gTTS(text=story_text, lang=lang_code, slow=style_choice==1)
        tts.save(temp_path)
        print(f"Successfully generated audio from your text.")

        # Determine speed adjustment factor
        speed_factor = 1.0  # Default neutral
        if style_choice == 1:
            speed_factor = 0.85  # Already slowed down during generation
        elif style_choice == 3:
            speed_factor = 1.3  # Energetic - speed up

        # Apply speed adjustment if needed for energetic style
        if style_choice == 3:
            # Load audio
            y, sr = librosa.load(temp_path)

            # Apply time stretching for speed change
            modified_audio = librosa.effects.time_stretch(y=y, rate=speed_factor)

            # Save the modified audio
            sf.write(final_path, modified_audio, sr)
            print(f"Applied {speed_factor}x speed adjustment for energetic style.")

            # Clean up temp file
            os.remove(temp_path)
        else:
            # Just rename the file if no speed adjustment needed
            os.rename(temp_path, final_path)

    except Exception as e:
        print(f"Error in speech generation: {e}")
        return

    # Save the story data as JSON for reference
    story_data = {
        "text": story_text,
        "style": ["Calm/Relaxed", "Neutral", "Energetic"][style_choice-1],
        "language": lang_code,
        "speed_factor": speed_factor
    }

    try:
        with open(f"voiceovers/{output_filename}.json", "w") as f:
            json.dump(story_data, f, indent=2)
        print(f"Story data saved to voiceovers/{output_filename}.json")
    except Exception as e:
        print(f"Could not save story data: {e}")

    print(f"\nVoiceover created successfully! Saved to {final_path}")
    print("\nTo use in your projects, simply play the MP3 file.")

if __name__ == "__main__":
    main()

==== AI-Driven Voiceover for Interactive Storytelling ====

Enter your story text (type 'END' on a new line when finished):
Hello This is Gen AI course for beginner
END

Select voice style:
1. Calm/Relaxed
2. Neutral
3. Energetic
Enter choice (1-3): 3

Select language:
1. English
2. Spanish
3. French
4. German
Enter language choice (1-5): 1

Enter output filename (without extension): test

Generating voiceover (this may take a moment)...
Processing text: "Hello This is Gen AI course for beginner"
Successfully generated audio from your text.
Applied 1.3x speed adjustment for energetic style.
Story data saved to voiceovers/test.json

Voiceover created successfully! Saved to voiceovers/test.mp3

To use in your projects, simply play the MP3 file.
